In [66]:
import sys
import itertools
import pandas as pd
import numpy as np
import nltk
import math

from google.cloud import storage
import google.cloud.storage

import _pickle as cPickle
#import gcsfs

Initialize the paths for data

In [67]:
rare_word = 50
stop_word = 1e4
unknown = 1

input_file = './data.txt'
vocab_file = './vocab.txt'
stop_file = './stop.txt'
vocab_pkl = './vocab.pkl'
#input_file = './data.txt'
#vocab_file = './vocab.txt'
#stop_file = './stop.txt'
#vocab_pkl = './vocab.pkl'

bucketname = "sdata-team60-paper168"

fs = None#gcsfs.GCSFileSystem(project='paper-168-team60')



In [68]:
def save_pkl(path, obj):
    #fs = gcsfs.GCSFileSystem(project='paper-168-team60')
    with open(path, 'wb') as f:
        cPickle.dump(obj, f)
        print(" [*] save %s" % path)

def load_pkl(path):
    with open(path) as f:
        obj = cPickle.load(f)
        print(" [*] load %s" % path)
    return obj

def save_npy(path, obj):
    np.save(path, obj)
    print(" [*] save %s" % path)

def load_npy(path):
    obj = np.load(path)
    print(" [*] load %s" % path)
    return obj


In [69]:
# configurations for the model, retaining same configurations as original code.
class Config:
    """feel free to play with these hyperparameters during training"""
    dataset = "resource"  # change this to the right data name
    data_path = "../%s" % dataset
    checkpoint_dir = "checkpoint"
    decay_rate = 0.95
    decay_step = 1000
    n_topics = 50
    learning_rate = 0.00002
    vocab_size = 619
    n_stops = 22 
    lda_vocab_size = vocab_size - n_stops
    n_hidden = 200
    n_layers = 2
    dropout = 1.0
    max_grad_norm = 1.0 #for gradient clipping
    total_epoch = 5
    init_scale = 0.075
    threshold = 0.5 #probability cut-off for predicting label to be 1
    forward_only = False #indicates whether we are in testing or training mode

In [70]:
#Reading the data and formating for futher preprocessing
def dump_vocab():
    df = pd.read_csv("./data.txt" ,dtype={"PID": int, "DAY_ID": int,"DX_GROUP_DESCRIPTION":"string","SERVICE_LOCATION": "string","OP_DATE":int }, sep="\t", header=0)
    #print(df[0:3])
    hist = df.groupby('DX_GROUP_DESCRIPTION').size().to_frame('SIZE').reset_index()
    #print(hist[0:3])
    #print("************************")
    hist_sort = hist.sort_values(by='SIZE', ascending=False)
    #print(hist_sort[0:3])
    #print("****************************************************************")
    count = hist.groupby('SIZE').size().to_frame('COUNT').reset_index()
    #print(count)

    ##Removing low frequency words

    hist = hist[hist['SIZE'] > rare_word]
    #print(hist)
    ##Remaining entries make the vocab anf formatting
    vocab = hist.sort_values(by='SIZE').reset_index()['DX_GROUP_DESCRIPTION']
    print(vocab)
    vocab.index += 2  # reserve 1 to unk
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucketname)
    bucket.blob("vocab.txt").upload_from_string(vocab.to_csv( sep='\t', header=False, index=True),'text')


    hist[hist['SIZE'] > stop_word].reset_index()['DX_GROUP_DESCRIPTION']\
            .to_csv(stop_file, sep='\t', header=False, index=False)

In [71]:
def load_vocab(fs):
    #Building the index keys for words

    word_to_index = {}

    with open(vocab_file, 'r') as f:
        line = f.readline()
        while line != '':
            #print(line)
            tokens = line.strip().split('\t')
            #print("tokens looks like")
            #print(tokens)
            #print("tokens[1] : ", tokens[1] )
            #print("tokens[0] : ", tokens[0])
            word_to_index[tokens[1]] = int(tokens[0])
            line = f.readline()
    save_pkl(vocab_pkl, {v: k for k, v in word_to_index.items()})
    print('dict size: ' + str(len(word_to_index)))
    return word_to_index

In [84]:
def convert_format(word_to_index, events):
    # order by PID, DAY_ID
    with open(input_file, mode='r') as f:
        # header
        header = f.readline().strip().split('\t')
        print(header)
        pos = {}
        for key, value in enumerate(header):
            pos[value] = key
        print(pos)

        docs = []
        doc = []
        sent = []
        labels = []
        label = []

        # init
        line = f.readline()
        tokens = line.strip().split('\t')
        pid = tokens[pos['PID']]
        day_id = tokens[pos['DAY_ID']]
        label.append(tag(events, pid, day_id))

        while line != '':
            tokens = line.strip().split('\t')
            c_pid = tokens[pos['PID']]
            c_day_id = tokens[pos['DAY_ID']]

            # closure
            if c_pid != pid:
                doc.append(sent)
                docs.append(doc)
                labels.append(label)
                sent = []
                doc = []
                label = []
                pid = c_pid
                day_id = c_day_id
                label.append(tag(events, pid, day_id))
            else:
                if c_day_id != day_id:
                    doc.append(sent)
                    sent = []
                    day_id = c_day_id
                    label.append(tag(events, pid, day_id))

            word = tokens[pos['DX_GROUP_DESCRIPTION']]
            try:
                sent.append(word_to_index[word])
            except KeyError:
                sent.append(unknown)

            line = f.readline()

        # closure
        doc.append(sent)
        docs.append(doc)
        labels.append(label)
        for i in range(len(labels[0])):
            print(doc[0][i] , "- " ,labels[0][i])
       

    return docs, labels

In [85]:
def split_data(docs, labels):
    # train, validate, test
    # X, Y,
    # TODO: YY
    print(len(docs))
    #print(docs)
    print(len(labels))
    #print(labels)

    save_pkl('./X_train.pkl', docs[:2000])
    save_pkl('./Y_train.pkl', labels[:2000])
    save_pkl('./X_valid.pkl', docs[2000:2700])
    save_pkl('./Y_valid.pkl', labels[2000:2700])
    save_pkl('./X_test.pkl', docs[2700:])
    save_pkl('./Y_test.pkl', labels[2700:])

In [86]:
def extract_events():
    # extract event "INPATIENT HOSPITAL"
    target_event = 'INPATIENT HOSPITAL'

    df = pd.read_csv(input_file, sep='\t', header=0)
    events = df[df['SERVICE_LOCATION'] == target_event]

    events = events.groupby(['PID', 'DAY_ID', 'SERVICE_LOCATION']).size().to_frame('COUNT').reset_index()\
        .sort_values(by=['PID', 'DAY_ID'], ascending=True)\
        .set_index('PID')

    return events

In [87]:
def tag(events, pid, day_id):
    return 1 if tag_logic(events, pid, day_id) else 0


def tag_logic(events, pid, day_id):
    try:
        patient = events.loc[int(pid)]

        # test whether have events within 30 days
        if isinstance(patient, pd.Series):
            return (int(day_id) <= patient.DAY_ID) & (patient.DAY_ID < int(day_id) + 30)

        return patient.loc[(int(day_id) <= patient.DAY_ID) & (patient.DAY_ID < int(day_id) + 30)].shape[0] > 0
    except KeyError:
        # the label is not in the [index]
        return False

In [88]:
def main():
    # dump_vocab()
    word_to_index = load_vocab(fs)
    events = extract_events()

    docs, labels = convert_format(word_to_index, events)
    split_data(docs, labels)

    # verify loading
main()

 [*] save ./vocab.pkl
dict size: 490
['PID', 'DAY_ID', 'DX_GROUP_DESCRIPTION', 'SERVICE_LOCATION', 'OP_DATE']
{'PID': 0, 'DAY_ID': 1, 'DX_GROUP_DESCRIPTION': 2, 'SERVICE_LOCATION': 3, 'OP_DATE': 4}
194 -  0


IndexError: list index out of range